In [ ]:
import os

In [ ]:
pwd

In [ ]:
os.chdir("../")

In [5]:
pwd

'E:\\Projects\\MLOPS\\kidney-disease-classification-mlops'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_dropout_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [7]:
from kidneyDiseaseClassifier import logger
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, model_name= "VGG16",  config_filepath= CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.model_name = model_name.upper()
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)[self.model_name]       # loading params for the specific model
        create_directories([self.config.artifacts_root])
        logger.info(f"Preparing Config-manager for {self.model_name} model.")
        
    def get_prepare_base_model_config(self) ->PrepareBaseModelConfig:
        base_model_config = self.config.prepare_base_model
        # param = self.params[self.model]  # loading params for the specific model
        
        print("Config ---> ", base_model_config)
        print("Params --> ",self.params)
        
        create_directories([base_model_config.root_dir])
        prepared_base_model_config = PrepareBaseModelConfig(
            root_dir= Path(base_model_config.root_dir),
            base_model_path= Path(base_model_config.base_model_path),
            updated_base_model_path= Path(base_model_config.updated_base_model_path),
            params_image_size= self.params.IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_dropout_rate= self.params.DROPOUT_RATE,
            params_weights= self.params.WEIGHTS,
            params_include_top= self.params.INCLUDE_TOP,
            params_classes= self.params.CLASSES
        )   
        
        return prepared_base_model_config

In [9]:
import os
import  urllib.request as request
from zipfile import  ZipFile
import tensorflow as tf

In [12]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.model = None
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path,model= self.model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, dropout_rate):    
        # Freeze the pretrained weights
        if freeze_all:
            for layer in model.layers:
                layer.trainable= False
                
            no_of_layers = len(model.layers)
            no_of_layers_to_train = int(no_of_layers/2)
    
            # Skipping the BN layers
            for layer in model.layers[-no_of_layers_to_train:]:
                if not isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = True
        
        elif (freeze_till is not None) and (freeze_till >0):
            for layer in model[:freeze_till]:
                if not isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = True
                else:
                    layer.trainable = False
        
        # Rebuild top
        x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rate, name="top_dropout")(x)

        prediction = tf.keras.layers.Dense(
            units= classes,
            activation='softmax',
            name="pred"
        )(x)
        
        full_model = tf.keras.models.Model(
            inputs= model.input,
            outputs = prediction
        )
        
        full_model.compile(
            optimizer= tf.keras.optimizers.SGD(learning_rate= learning_rate, momentum=0.9),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        full_model.summary()
        logger.info(f"Loaded the base model.")
        return  full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes= self.config.params_classes,
            freeze_all= True,
            freeze_till= None,
            learning_rate= self.config.params_learning_rate,
            dropout_rate= self.config.params_dropout_rate
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
        logger.info(f"Final model is saving at {self.config.updated_base_model_path}")

In [13]:
try:
    config = ConfigurationManager(model_name='VGG16')
    prepare_base_model_config = config.get_prepare_base_model_config()
    
    prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-12-10 20:24:03,877]: INFO: common: yaml file: config\config.yaml loaded successfully!
[2023-12-10 20:24:03,881]: INFO: common: yaml file: params.yaml loaded successfully!
[2023-12-10 20:24:03,882]: INFO: common: Created directory at : artifacts
[2023-12-10 20:24:03,884]: INFO: 1631773706: Preparing Config-manager for VGG16 model.
Config --->  {'root_dir': 'artifacts/base_model', 'base_model_path': 'artifacts/base_model/base_model.h5', 'updated_base_model_path': 'artifacts/base_model/base_model_updated.h5'}
Params -->  {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 15, 'CLASSES': 4, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01, 'DROPOUT_RATE': 0.3}
[2023-12-10 20:24:03,887]: INFO: common: Created directory at : artifacts/base_model
[2023-12-10 20:24:04,392]: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the mode